In [2]:
from dataclasses import dataclass
from vi import Agent, Config, Simulation, Vector2
import pygame as pg
import os

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
frame_dir = "frames"
os.makedirs(frame_dir, exist_ok=True)
frame_count = 0

def save_frame(screen):
    global frame_count
    pg.image.save(screen, os.path.join(frame_dir, f"frame_{frame_count:05d}.png"))
    frame_count += 1

In [4]:
class RecordingSimulation(Simulation):
    def __init__(self, config):
        super().__init__(config)
        self.frame_count = 0
        os.makedirs("frames", exist_ok=True)

    def after_update(self) -> None:
        # Draw everything to the screen
        self._all.draw(self._screen)

        if self.config.visualise_chunks:
            self.__visualise_chunks()

        # Save current frame as an image
        pg.image.save(self._screen, f"frames/frame_{self.frame_count:05d}.png")

        # Update the screen with the new image
        pg.display.flip()

        self._clock.tick(self.config.fps_limit)

        current_fps = self._clock.get_fps()
        if current_fps > 0:
            self._metrics.fps._push(current_fps)

            if self.config.print_fps:
                print(f"FPS: {current_fps:.1f}")  # noqa: T201

        lonely_count = 0
        for agent in self._all:  # assumes self._all is a list or container of agents
            if not list(agent.in_proximity_accuracy()):
                lonely_count += 1

        if lonely_count == 0:
            print(f"✅ Frame {self.frame_count}: All agents have neighbors — full swarm achieved.")
            self.config.swarm_formed = True
        else:
            print(f"⚠️ Frame {self.frame_count}: {lonely_count} lonely agents.")
        
        # Increment a frame counter (you may need to initialize it somewhere)
        self.frame_count += 1


In [13]:
@dataclass
class FlockingConfig(Config):
    alignment_weight: float = 1.0
    cohesion_weight: float = 0.04
    separation_weight: float = 2.85
    swarm_formed: bool = False

class FlockingAgent(Agent):
    def on_spawn(self):
        self.lonely_multiplier = 5

    def change_position(self):
        self.there_is_no_escape()
        neighbors = list(self.in_proximity_accuracy())
        if not neighbors and not(self.config.swarm_formed):
            self.pos += self.move * self.lonely_multiplier
            return
        elif not neighbors:
            self.pos += self.move
            return
        
        separationVelocity = Vector2() # move away from nearby agents
        alignmentVelocity = Vector2() # match the average direction of nearby agents
        cohesionVelocity = Vector2() # move toward the average position of nearby agents
        num_neighbors = len(neighbors)

        for neighbor, distance in neighbors: # loop trough each nearby agent.
            offset = self.pos - neighbor.pos # a vector pointing away from the neighbor
            if distance > 0:
                separationVelocity += offset / (distance**1.3) # push neighbors away

            alignmentVelocity += neighbor.move # sum al neighbors movement direction
            cohesionVelocity += neighbor.pos # sum all neigbors position

        # take the average of each vector to get a single direction fro each behavior
        separationVelocity /= num_neighbors
        alignmentVelocity /= num_neighbors
        cohesionVelocity = (cohesionVelocity / num_neighbors - self.pos)

        # multiply bt weights
        separationVelocity *= self.config.separation_weight
        alignmentVelocity *= self.config.alignment_weight
        cohesionVelocity *= self.config.cohesion_weight

        # single movement vector for all behaviors
        self.move = (separationVelocity + alignmentVelocity + cohesionVelocity).normalize() * self.config.movement_speed
        self.pos += self.move

(
    RecordingSimulation(
        FlockingConfig(image_rotation = True, movement_speed = 5.0, radius = 50, seed = 777, duration = 10000, fps_limit = 60)
    )
    .batch_spawn_agents(100, FlockingAgent, images=["images/triangle.png"])
    .run()
)

⚠️ Frame 0: 33 lonely agents.
⚠️ Frame 1: 24 lonely agents.
⚠️ Frame 2: 14 lonely agents.
⚠️ Frame 3: 5 lonely agents.
⚠️ Frame 4: 6 lonely agents.
⚠️ Frame 5: 6 lonely agents.
⚠️ Frame 6: 4 lonely agents.
⚠️ Frame 7: 2 lonely agents.
⚠️ Frame 8: 1 lonely agents.
⚠️ Frame 9: 1 lonely agents.
⚠️ Frame 10: 1 lonely agents.
⚠️ Frame 11: 1 lonely agents.
⚠️ Frame 12: 2 lonely agents.
⚠️ Frame 13: 1 lonely agents.
⚠️ Frame 14: 1 lonely agents.
⚠️ Frame 15: 2 lonely agents.
⚠️ Frame 16: 1 lonely agents.
⚠️ Frame 17: 1 lonely agents.
⚠️ Frame 18: 1 lonely agents.
⚠️ Frame 19: 1 lonely agents.
✅ Frame 20: All agents have neighbors — full swarm achieved.
⚠️ Frame 21: 2 lonely agents.
⚠️ Frame 22: 2 lonely agents.
⚠️ Frame 23: 3 lonely agents.
⚠️ Frame 24: 3 lonely agents.
⚠️ Frame 25: 4 lonely agents.
⚠️ Frame 26: 3 lonely agents.
⚠️ Frame 27: 1 lonely agents.
⚠️ Frame 28: 2 lonely agents.
⚠️ Frame 29: 2 lonely agents.
⚠️ Frame 30: 3 lonely agents.
⚠️ Frame 31: 4 lonely agents.
⚠️ Frame 32: 4 l

KeyboardInterrupt: 